In [1]:
#install libraries
!pip install python-dotenv
!pip install pymongo[srv]
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#import necessary modules
from dotenv import load_dotenv
from datetime import datetime
from pprint import pprint
import pandas as pd
import os
import bson
import pymongo

load_dotenv()

True

In [3]:
print(pymongo.__version__)

4.3.3


In [4]:
#connect to mongoDB

client = pymongo.MongoClient(os.environ.get("CONN_STRING"))

In [5]:
print(client.primary)

('ac-tzhuuxz-shard-00-01.edbfm1z.mongodb.net', 27017)


In [6]:
print(client.secondaries)

{('ac-tzhuuxz-shard-00-02.edbfm1z.mongodb.net', 27017), ('ac-tzhuuxz-shard-00-00.edbfm1z.mongodb.net', 27017)}


In [7]:
client.list_database_names()

['ETA_Trucks', 'admin', 'local']

In [5]:
#connect to Database

db=client['ETA_Trucks']

In [9]:
db.list_collection_names()

['truck_schedule_details',
 'truck_details',
 'driver_details',
 'route_details',
 'city_details']

In [10]:
#find one

filter={'name':{"$eq":"Alabama"}}
db.city_details.find_one(filter)

{'_id': ObjectId('63cf28c750cce1d3ca60f9f9'),
 'city_id': 1,
 'name': 'Alabama',
 'lat': 42.998,
 'lon': -78.185}

In [11]:
#find using _id

filter={"_id":bson.ObjectId('63cf28c750cce1d3ca60f9fe')}
db.city_details.find_one(filter)

{'_id': ObjectId('63cf28c750cce1d3ca60f9fe'),
 'city_id': 6,
 'name': 'Colorado Springs',
 'lat': 38.834,
 'lon': -104.821}

In [13]:
#find - implicit And - find most efficient diesel trucks with age greate than 12


filter={"fuel_type":'diesel',
        "truck_age":{"$gt":12}}

projection={"_id":0,"fuel_type":0}

sort_seq=[("truck_age",pymongo.ASCENDING),
          ("load_capacity_pounds",pymongo.DESCENDING),
          ("mileage_mpg",pymongo.DESCENDING)]

result=db.truck_details.find(filter, projection).sort(sort_seq).limit(5)

for i in result:
  pprint(i)
  print("\n")

{'load_capacity_pounds': 20000,
 'mileage_mpg': 29,
 'truck_age': 13,
 'truck_id': '7590026a0d2a4c1d8bfe7426765c260b'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '4d8ed7b8fab346b1b057a21f332a343e'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '55c5b1edae2c47e6a4e4bade3835f610'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '0ad2682b55b84d91868d9528226f3aaf'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '7b74e1f38e694880b4723fe38b396fb0'}


CPU times: user 6.54 ms, sys: 1.26 ms, total: 7.8 ms
Wall time: 394 ms


In [23]:
#count no.of documents - young proactive male drivers who are reliable for long distances

filter={
    "$and":[
        {"gender":"male"},
        {"experience":{"$gt":10}},
        {"driving_style":{"$in": ["proactive"]}}
    ],
    "$or":[
        {"$and":[
            {"age":{"$gte":35}},
            {"age":{"$lte":45}}
        ]},
        {"ratings":{"$gt":4}}

    ]
}

print(db.driver_details.count_documents(filter))

sort_seq=[("ratings",-1),
          ("age",1)]
result=db.driver_details.find(filter).sort(sort_seq).limit(5)

for i in result:
  pprint(i)
  print("\n")

20389
{'_id': ObjectId('63cf2c1050cce1d3ca66fae2'),
 'age': 39,
 'driver_id': '76a283a440aa4799a87086b24a65f1db',
 'driving_style': 'proactive',
 'experience': 11,
 'gender': 'male',
 'name': 'Michael Mcconnell',
 'ratings': 5,
 'vehicle_no': 'a7022bf811ed40ef911d775c9da1ca9e'}


{'_id': ObjectId('63cf2c0650cce1d3ca66c547'),
 'age': 40,
 'driver_id': '046ad46921104e6db54d1a2399a7944f',
 'driving_style': 'proactive',
 'experience': 11,
 'gender': 'male',
 'name': 'Andrew Forbes',
 'ratings': 5,
 'vehicle_no': '254869b1ffac41feae6790037ed12844'}


{'_id': ObjectId('63cf2bec50cce1d3ca662b18'),
 'age': 40,
 'driver_id': '843f66e8bcf2479e9997a50603cb7a97',
 'driving_style': 'proactive',
 'experience': 11,
 'gender': 'male',
 'name': 'Scott Johnson',
 'ratings': 5,
 'vehicle_no': '1efbcb288cb843c4bc0ff7b941ca5b70'}


{'_id': ObjectId('63cf2b8550cce1d3ca64148c'),
 'age': 41,
 'driver_id': '4780d7706490482ca19e4f5ed1dbba81',
 'driving_style': 'proactive',
 'experience': 22,
 'gender': 'male',


In [24]:
#regex- first name 'Tyler'

filter={"name":{"$regex":"^Tyler .*", "$options":'ms'}}
result=db.driver_details.find(filter).limit(2)

# 'm' for caret(^)-startWith and $- endWith
# 's' for period(.) - all characters

for i in result:
  pprint(i)

{'_id': ObjectId('63cf2b8450cce1d3ca6410a1'),
 'age': 41,
 'driver_id': '40287c65b040442f899c0affbcad207a',
 'driving_style': 'proactive',
 'experience': 22,
 'gender': 'male',
 'name': 'Tyler Saunders',
 'ratings': 3.3,
 'vehicle_no': '765d4022c30848c3a4517f003d2689e7'}
{'_id': ObjectId('63cf2b8450cce1d3ca6410ff'),
 'age': 51,
 'driver_id': 'd0f65590f84c42a697db283a6795666c',
 'driving_style': 'proactive',
 'experience': 12,
 'gender': 'male',
 'name': 'Tyler Johnson',
 'ratings': 2.9,
 'vehicle_no': 'f1080c451a90403c97e2db8699c26c3f'}


In [25]:
#regex- last name 'Dean'

filter={"name":{"$regex":".* Dean$", "$options":'ms'}}
result=db.driver_details.find(filter).limit(2)

for i in result:
  pprint(i)

{'_id': ObjectId('63cf2b8450cce1d3ca64138c'),
 'age': 44,
 'driver_id': 'e7bb0b7942994e7d8eed9a8639e63713',
 'driving_style': 'conservative',
 'experience': 14,
 'gender': 'male',
 'name': 'Tyler Dean',
 'ratings': 4.4,
 'vehicle_no': 'ccd30d82a450492884cd0ce67814ea49'}
{'_id': ObjectId('63cf2b8550cce1d3ca641704'),
 'age': 46,
 'driver_id': '7bef3146c7ee4a7593e013334e0d2cd2',
 'driving_style': 'proactive',
 'experience': 14,
 'gender': 'male',
 'name': 'Robert Dean',
 'ratings': 1.5,
 'vehicle_no': 'a3604250716849caad7fb71f40872d59'}


In [26]:
#update one
filter={"truck_id":"2f0705c938e545a0aa03c505ec97751d"}

update={
    "$set":
    {"fines":[
        {"date":datetime.utcnow(),
         "amount":500,
         "reason":"Speed Limit",
         "driver_id":"c3173876aeac4604863da23d0a90f6f3"}
         ]}}
         
db.truck_details.update_one(filter,update)

In [27]:
#update to list
filter={"truck_id":"2f0705c938e545a0aa03c505ec97751d"}

update={
    "$push":
    {"fines":
     {"date":datetime.utcnow(),
      "amount":5000,
      "reason":"Red Light Crossed",
      "driver_id":"c3173876aeac4604863da23d0a90f6f3"}}}

db.truck_details.update_one(filter,update)

In [28]:
#update many - increment truck age for all trucks
filter={}

update={"$inc":{"truck_age":1}}

db.truck_details.update_many(filter,update)

In [29]:
#update many- decrement truck mileage
filter={"fuel_type":"gas"}

update={"$inc":{"mileage_mpg":-2}}

db.truck_details.update_many(filter,update)

In [15]:
#upsert
filter={"name":"Houston"}

update={"$set":{
    "name":"Houston",
    "city_id":50,
    "lat":29.74,
    "lon":-95.46}}

db.city_details.update_many(filter,update,upsert=True)

In [16]:
#delete
db.city_details.delete_many({"name":"Houston"})

In [17]:
#without indexes
pprint(db.truck_schedule_details.find({"origin":"Alabama"}).sort("date",pymongo.DESCENDING).explain())

{'$clusterTime': {'clusterTime': Timestamp(1674618400, 1),
                  'signature': {'hash': b'\xa8\xe0\x08\xac\x9e\xb4\xf0\x87'
                                        b'\xc4\xa0\xfe\x00\x16<\td=f\x8c\x7f',
                                'keyId': 7154063244682854402}},
 'command': {'$db': 'ETA_Trucks',
             'filter': {'origin': 'Alabama'},
             'find': 'truck_schedule_details',
             'sort': {'date': -1}},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'advanced': 13620,
                                        'executionTimeMillisEstimate': 51,
                                        'inputStage': {'advanced': 13620,
                                                       'direction': 'forward',
                                                       'docsExamined': 566110,
                                                       'executionTimeMillisEstimate': 50,
                                                       'fi

In [18]:
#create indexes
from pymongo import IndexModel, ASCENDING, DESCENDING

index1 = IndexModel([("origin", ASCENDING), ("date", DESCENDING)]) #for unique indexes, pass unique=True
index2 = IndexModel([("destination", ASCENDING)])

db.truck_schedule_details.create_indexes([index1,index2])

['origin_1_date_-1', 'destination_1']

In [19]:
#with indexes
pprint(db.truck_schedule_details.find({"origin":"Alabama"}).sort("date",pymongo.DESCENDING).explain())

{'$clusterTime': {'clusterTime': Timestamp(1674619148, 3),
                  'signature': {'hash': b'\x05\xa5\x8f\xe4\x97\x03\x1a\xc7'
                                        b'/\x9a\xd1\x89\xe3aXeC\xb6\xe9~',
                                'keyId': 7154063244682854402}},
 'command': {'$db': 'ETA_Trucks',
             'filter': {'origin': 'Alabama'},
             'find': 'truck_schedule_details',
             'sort': {'date': -1}},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'advanced': 13620,
                                        'alreadyHasObj': 0,
                                        'docsExamined': 13620,
                                        'executionTimeMillisEstimate': 36,
                                        'inputStage': {'advanced': 13620,
                                                       'direction': 'forward',
                                                       'dupsDropped': 0,
                                   

In [20]:
db.truck_schedule_details.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'origin_1_date_-1': {'v': 2, 'key': [('origin', 1), ('date', -1)]},
 'destination_1': {'v': 2, 'key': [('destination', 1)]}}

In [7]:
db2=client['Ecom']

In [9]:


db2.Quant.insert_many([
    {"item":"abc","quantity":41,"lastModified":datetime.strptime("2023-01-09 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"item":"def","quantity":56,"lastModified":datetime.strptime("2023-01-09 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"item":"xyz","quantity":78,"lastModified":datetime.strptime("2022-02-09 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"item":"jkl","quantity":67,"lastModified":datetime.strptime("2023-01-09 01:01:01", "%Y-%m-%d %H:%M:%S")}

]
)

db2.sales.insert_many([
    {"_id":1,"item":"abc","price":8,"quantity":4,"date":datetime.strptime("2022-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id":2,"item":"xyz","price":8,"quantity":4,"date":datetime.strptime("2022-02-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id":3,"item":"jkl","price":10,"quantity":5,"date":datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id":4,"item":"abc","price":12,"quantity":6,"date":datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id":5,"item":"def","price":16,"quantity":8,"date":datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id":6,"item":"jkl","price":10,"quantity":5,"date":datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id":7,"item":"abc","price":10,"quantity":5,"date":datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")}
])

In [17]:
#transactions

def callback(session, sales_id, item, quantity, price, date):

  sales=session.client.Ecom.sales
  quant=session.client.Ecom.Quant

  #Transaction Operations 

  sales.insert_one({"_id":sales_id, 
                    "item":item, 
                    "price":price, 
                    "quantity":quantity, 
                    "date":date}, 
                   session=session)
  t=6/0
  quant.update_one({"item":item},{
      "$inc":{
                "quantity": -abs(quantity)
             },
      "$currentDate":{"lastModified":True}
  }, session=session)

In [18]:
with client.start_session() as session:
  session.with_transaction(
      lambda a : callback(a, sales_id=10, item='xyz', quantity=10, price=20, date=datetime.now())
  )


In [21]:
stage_1={"$lookup":
     {
     "from":"sales",
     "localField":"item",
     "foreignField":"item",
     "as":"sales_list"}
    }


stage_2={
        "$unset":"sales_list._id"
            }


stage_3= {"$merge":{
        "into":{"db":"Ecom","coll":"Inventory"}
    }}

results=db2.Quant.aggregate([stage_1,stage_2,stage_3])

for i in results:
  pprint(i)
  print("\n")

In [29]:
filter={
    "sales_list":{"$elemMatch":
                  {"quantity":
                   {"$ne":4},
                   "$and":[
                       {"date":{"$gte":datetime(2022,1,1)}},
                       {"date":{"$lte":datetime(2023,2,1)}}
                       ]
                   }}
}

result=db2.Inventory.find(filter)


for i in result:
  pprint(i)
  print('\n')

{'_id': ObjectId('63d2f5b461859bb164d0db8b'),
 'item': 'abc',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 41,
 'sales_list': [{'date': datetime.datetime(2022, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 8,
                 'quantity': 4},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 12,
                 'quantity': 6},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 10,
                 'quantity': 5}]}


{'_id': ObjectId('63d2f5b461859bb164d0db8c'),
 'item': 'def',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 56,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'def',
                 'price': 16,
                 'quantity': 8}]}


{'_id': ObjectId('63d2f5b461859bb164d0db8d'),
 'item': 'xyz',
 'lastModified': date

In [30]:

result=db2.Inventory.find({
    "sales_list.quantity":{"$ne":4},
     "sales_list.date":{"$gte":datetime(2022,1,1)},
    "sales_list.date":{"$lte":datetime(2023,2,1)}
              
})

for i in result:
  pprint(i)
  print('\n')

{'_id': ObjectId('63d2f5b461859bb164d0db8c'),
 'item': 'def',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 56,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'def',
                 'price': 16,
                 'quantity': 8}]}


{'_id': ObjectId('63d2f5b461859bb164d0db8e'),
 'item': 'jkl',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 67,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'jkl',
                 'price': 10,
                 'quantity': 5},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'jkl',
                 'price': 10,
                 'quantity': 5}]}




In [31]:
filter={"item":"xyz"}

update={  "$inc":{ 
    "quantity": -7
      },
      "$push":{
                "sales_list":{
                 "item":"xyz",
                 "price":14,
                 "quantity":7,
                  "date":datetime.now()
                     }
             },
           "$currentDate":{"lastModified":True}
              }


db2.Inventory.update_many(filter,update)


In [8]:
stage_1= {
      "$set": {
         "VAT": {
           "$map": {
             "input": "$sales_list",
             "as": "grade",
             "in": { "$multiply": ["$$grade.price", 0.05 ] }
           }
         }
      }
   }

result=db2.Inventory.aggregate([stage_1])

for i in result:
  pprint(i)

{'VAT': [0.4, 0.6000000000000001, 0.5],
 '_id': ObjectId('63d2f5b461859bb164d0db8b'),
 'item': 'abc',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 41,
 'sales_list': [{'date': datetime.datetime(2022, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 8,
                 'quantity': 4},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 12,
                 'quantity': 6},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 10,
                 'quantity': 5}]}
{'VAT': [0.8],
 '_id': ObjectId('63d2f5b461859bb164d0db8c'),
 'item': 'def',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 56,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'def',
                 'price': 16,
                 'quantity': 8}]}
{'VAT': [0.4, 1.0, 1.0, 0.700000

In [16]:
# find Avg age of all drivers

stage_1= {'$group': {'_id': None, 
                     'avg_Age': {'$avg': '$age'}
                     }
          }

r=db.driver_details.aggregate([stage_1])

for i in r:
  pprint(i)

In [16]:
# find Avg age of all drivers by All documents, Gender, driving style in one pipeline

facet_stage={'$facet': {
            'AVG_AGE_ALL': [{'$group': {'_id': None,
                                        'avg_Age': {'$avg': '$age'},
                                        'total_count': {'$sum': 1}}}],
                        
            'AVG_AGE_GENDER': [{'$group': {'_id': '$gender',
                                           'avg_Age': {'$avg': '$age'},
                                           'total_count': {'$sum': 1}}}],
                        
            'AVG_AGE_STYLE': [{'$group': {'_id': '$driving_style',
                                          'avg_Age': {'$avg': '$age'},
                                          'total_count': {'$count': {}}}}]
              }}

r=db.driver_details.aggregate([facet_stage])

for i in r:
  pprint(i)

In [32]:
#join truck and driver details, add total_trucks and total_fines fields and save to another collection

stage_1={"$lookup":
                {
                "from":"truck_details",
                "localField":"vehicle_no",
                "foreignField":"truck_id",
                "as":"trucks_assigned"}
              }

stage_2={"$unset":["vehicle_no","trucks_assigned._id"]}

stage_3={'$addFields': {'t_trucks': {'$size': '$trucks_assigned'}}}

stage_4={"$unwind":"$trucks_assigned"}

stage_5={'$addFields': {'t_fines': {'$cond': {'else': None,
                                      'if': {'$and': [{'$isArray': '$trucks_assigned.fines'}]},
                                      'then': {'$size': '$trucks_assigned.fines'}}}}}

stage_6={'$out': {'coll': 'driver_trucks', 'db': 'ETA_Trucks'}}

limit_stage={"$limit":5}

temp_stage={"$match":{"name":"Scott Hawkins"}}

result=db.driver_details.aggregate([stage_1, stage_2, stage_3, stage_4, stage_5, limit_stage, stage_6])


#NOTE- $out and $merge are not optimal compute wise to store large number of documents from an aggregation pipeline

for i in result:
  pprint(i)
  print("\n")

In [37]:
#add a GeoJSON field
stage_1={'$addFields': {'location': {'coordinates': ['$lon', '$lat'], 'type': 'Point'}}}

stage_2={'$merge': {
            'into': 'city_details', 
            'on': '_id', 
            'whenMatched': 'replace'}
         }

result=db.city_details.aggregate([stage_1, stage_2])

for i in result:
  pprint(i)
  print("\n")

In [38]:
#create index for location
from pymongo import IndexModel, ASCENDING, DESCENDING

index1 = IndexModel([("location", "2dsphere")]) #for unique indexes, pass unique=True

db.city_details.create_indexes([index1])

['location_2dsphere']

In [41]:
#find nearest centers for delivery from a given set of coordinates

result=db.city_details.aggregate([
    
    {'$geoNear': {'distanceField': 'dist.calculated',
              'distanceMultiplier': 0.001,
              'includeLocs': 'dist.location',
              'maxDistance': 800000,
              'minDistance': 0,
              'near': {'coordinates': [-97.617134, 30.222296], 'type': 'Point'},
              'spherical': True}}
])

for i in result:
  pprint(i)
  print("\n")

{'_id': ObjectId('63cf28c750cce1d3ca60fa11'),
 'city_id': 25,
 'dist': {'calculated': 211.57425184978044,
          'location': {'coordinates': [-95.538, 29.618], 'type': 'Point'}},
 'lat': 29.618,
 'location': {'coordinates': [-95.538, 29.618], 'type': 'Point'},
 'lon': -95.538,
 'name': 'Missouri City'}


{'_id': ObjectId('63cf28c750cce1d3ca60fa22'),
 'city_id': 42,
 'dist': {'calculated': 278.27607088631123,
          'location': {'coordinates': [-94.903, 29.384], 'type': 'Point'}},
 'lat': 29.384,
 'location': {'coordinates': [-94.903, 29.384], 'type': 'Point'},
 'lon': -94.903,
 'name': 'Texas City'}


{'_id': ObjectId('63cf28c750cce1d3ca60fa1c'),
 'city_id': 36,
 'dist': {'calculated': 584.0221857132893,
          'location': {'coordinates': [-97.516, 35.468], 'type': 'Point'}},
 'lat': 35.468,
 'location': {'coordinates': [-97.516, 35.468], 'type': 'Point'},
 'lon': -97.516,
 'name': 'Oklahoma City'}


{'_id': ObjectId('63cf28c750cce1d3ca60f9fc'),
 'city_id': 4,
 'dist': {'calcu

In [7]:
#total delayed journeys count

stage_1={'$group': {
            '_id': {
                '$toBool': {
                    '$convert': {
                        'input': '$delay',
                        'to': 'decimal'}}},
            'totalcount': {'$sum': 1}}
         }

stage_2={'$project': {
          '_id': 0, 
          'is_delay': '$_id', 
          'totalcount': '$totalcount'}
         }


result=db.truck_schedule_details.aggregate([stage_1, stage_2])


for i in result:
  pprint(i)
  print("\n")

{'is_delay': True, 'totalcount': 37113}


{'is_delay': False, 'totalcount': 528997}




In [12]:
#find the most congested routes

stage_1={'$match': {'delay': '1'}}

stage_2={'$group': {'_id': {'destination': '$destination', 'origin': '$origin'},
            'route_id': {'$first': '$route_id'},
            'totalcount': {'$sum': 1}}}

stage_3={'$project': {'_id': 0,
              'route': '$_id',
              'route_id': '$route_id',
              'total_delays': '$totalcount'}}

stage_4={'$sort': {'total_delays': -1}}

limit_stage={"$limit":10}

result=db.truck_schedule_details.aggregate([stage_1, stage_2, stage_3, stage_4, limit_stage])


for i in result:
  pprint(i)
  print("\n")

{'route': {'destination': 'Pennsylvania Furnace',
           'origin': 'South Dakota Park'},
 'route_id': 'South_Dakota_Park_Pennsylvania_Furnace_97381',
 'total_delays': 104}


{'route': {'destination': 'South Dakota Park',
           'origin': 'Pennsylvania Furnace'},
 'route_id': 'Pennsylvania_Furnace_South_Dakota_Park_1751',
 'total_delays': 104}


{'route': {'destination': 'Nebraska City', 'origin': 'Pennsylvania Furnace'},
 'route_id': 'Pennsylvania_Furnace_Nebraska_City_1193',
 'total_delays': 99}


{'route': {'destination': 'Pennsylvania Furnace', 'origin': 'Nebraska City'},
 'route_id': 'Nebraska_City_Pennsylvania_Furnace_79948',
 'total_delays': 99}


{'route': {'destination': 'Nevada', 'origin': 'Worcester'},
 'route_id': 'Worcester_Nevada_86544',
 'total_delays': 95}


{'route': {'destination': 'Worcester', 'origin': 'Nevada'},
 'route_id': 'Nevada_Worcester_59645',
 'total_delays': 95}


{'route': {'destination': 'Pennsylvania Furnace', 'origin': 'Oklahoma City'},
 'route_

In [14]:
# find which months have the most delays and total routes affected each month for past two years

stage_1={'$match': {'delay': '1'}}

stage_2={'$group': {'_id': {'month': {'$month': '$date'}, 'year': {'$year': '$date'}},
            'routes': {'$addToSet': '$route_id'},
            'total_delays': {'$sum': 1}}}

stage_3={'$project': {'_id': 0,
              'month_year': '$_id',
              'total_delays': '$total_delays',
              'unique_routes_affected': {'$size': '$routes'}}}

stage_4={'$sort': {'total_delays': -1}}

result=db.truck_schedule_details.aggregate([stage_1, stage_2, stage_3, stage_4])


for i in result:
  pprint(i)
  print("\n")

{'month_year': {'month': 1, 'year': 2017},
 'total_delays': 3740,
 'unique_routes_affected': 1404}


{'month_year': {'month': 3, 'year': 2017},
 'total_delays': 2495,
 'unique_routes_affected': 1208}


{'month_year': {'month': 12, 'year': 2017},
 'total_delays': 2339,
 'unique_routes_affected': 1172}


{'month_year': {'month': 1, 'year': 2018},
 'total_delays': 1826,
 'unique_routes_affected': 1009}


{'month_year': {'month': 3, 'year': 2018},
 'total_delays': 1818,
 'unique_routes_affected': 1042}


{'month_year': {'month': 2, 'year': 2018},
 'total_delays': 1797,
 'unique_routes_affected': 1025}


{'month_year': {'month': 11, 'year': 2018},
 'total_delays': 1735,
 'unique_routes_affected': 975}


{'month_year': {'month': 9, 'year': 2018},
 'total_delays': 1708,
 'unique_routes_affected': 975}


{'month_year': {'month': 4, 'year': 2018},
 'total_delays': 1677,
 'unique_routes_affected': 1062}


{'month_year': {'month': 12, 'year': 2018},
 'total_delays': 1600,
 'unique_routes_affected

In [17]:
result=db.truck_schedule_details.aggregate([
    {
        "$match":{
            "delay":"1"
        }
    },
    {
        "$group":{
            "_id":{
                "year":{"$year":"$date"},
                "month":{"$month":"$date"},
                "route":"$route_id"
            },
            "total_delays_route_wise":{"$count":{}}
        }
    },
    {
        "$group":{
            "_id":{
                "year":"$_id.year",
                "month":"$_id.month"
            },
            "total_delays":{"$sum":"$total_delays_route_wise"},
            "unique_routes_affected":{"$count":{}}
        }
    },
    {
      "$sort":{"total_delays":-1}  
    },

])

for i in result:
  pprint(i)
  print("\n")

{'_id': {'month': 1, 'year': 2017},
 'total_delays': 3740,
 'unique_routes_affected': 1404}


{'_id': {'month': 3, 'year': 2017},
 'total_delays': 2495,
 'unique_routes_affected': 1208}


{'_id': {'month': 12, 'year': 2017},
 'total_delays': 2339,
 'unique_routes_affected': 1172}


{'_id': {'month': 1, 'year': 2018},
 'total_delays': 1826,
 'unique_routes_affected': 1009}


{'_id': {'month': 3, 'year': 2018},
 'total_delays': 1818,
 'unique_routes_affected': 1042}


{'_id': {'month': 2, 'year': 2018},
 'total_delays': 1797,
 'unique_routes_affected': 1025}


{'_id': {'month': 11, 'year': 2018},
 'total_delays': 1735,
 'unique_routes_affected': 975}


{'_id': {'month': 9, 'year': 2018},
 'total_delays': 1708,
 'unique_routes_affected': 975}


{'_id': {'month': 4, 'year': 2018},
 'total_delays': 1677,
 'unique_routes_affected': 1062}


{'_id': {'month': 12, 'year': 2018},
 'total_delays': 1600,
 'unique_routes_affected': 933}


{'_id': {'month': 5, 'year': 2017},
 'total_delays': 1571,
 